In [1]:
# Import libraries
import os, getpass, glob


import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms as mtran
import healpy as hp

from collections import defaultdict

In [2]:
# Define a function for weighted medians, or weighted quantiles
def weighted_quantiles(values, weights, quantiles=0.5):
    if len(values) == 0:
        return 0
    else:
        i = np.argsort(values)
        c = np.cumsum(weights[i])
        return values[i[np.searchsorted(c, np.array(quantiles) * c[-1])]]

In [3]:
MEDIANS = {
    'SIBYLL2.1': {
        'Tier 1': {
            '2012': {
                'median': None,
                'error': None
            }
        },
        'Tier 2': {
            '2012': {
                'median': None,
                'error': None
            }
        },
        'Tier 3': {
            '2012': {
                'median': None,
                'error': None
            },
            '2015': {
                'median': None,
                'error': None
            }
        },
        'Tier 4': {
            '2012': {
                'median': None,
                'error': None
            },
            '2015': {
                'median': None,
                'error': None
            }
        }
    },
    'EPOS-LHC': {
        'Tier 1': {
            '2012': {
                'median': None,
                'error': None
            }
        },
        'Tier 2': {
            '2012': {
                'median': None,
                'error': None
            }
        },
        'Tier 3': {
            '2012': {
                'median': None,
                'error': None
            },
            '2018': {
                'median': None,
                'error': None
            }
        },
        'Tier 4': {
            '2012': {
                'median': None,
                'error': None
            },
            '2018': {
                'median': None,
                'error': None
            }
        }
    },
    'QGSJET-II-04': {
        'Tier 1': {
            '2012': {
                'median': None,
                'error': None
            }
        },
        'Tier 2': {
            '2012': {
                'median': None,
                'error': None
            }
        },
        'Tier 3': {
            '2012': {
                'median': None,
                'error': None
            },
            '2018': {
                'median': None,
                'error': None
            }
        },
        'Tier 4': {
            '2012': {
                'median': None,
                'error': None
            },
            '2018': {
                'median': None,
                'error': None
            }
        }
    },
    'SIBYLL2.3': {
        'Tier 1': {
            '2012': {
                'median': None,
                'error': None
            }
        },
        'Tier 2': {
            '2012': {
                'median': None,
                'error': None
            }
        },
        'Tier 3': {
            '2012': {
                'median': None,
                'error': None
            }
        },
        'Tier 4': {
            '2012': {
                'median': None,
                'error': None
            }
        }
    },
    'SIBYLL2.3d': {
        'Tier 3': {
            '2018': {
                'median': None,
                'error': None
            }
        },
        'Tier 4': {
            '2018': {
                'median': None,
                'error': None
            }
        }
    }
}

In [ ]:
for model, MTIERS in MEDIANS.items():
    print(model)
    print('--------------------------------')
    for mtier, YEARS in MTIERS.items():
        for year in YEARS.keys():
            print(year)
    
            # Load in the particle sims - note that these will also be your variable names
            KEYS = ['energy', 'hits', 'reco_pass', 'showerplane_zen', 'laputop_zen', 'Hweights', 'Gweights']
            for key in KEYS:
                if os.path.isfile(f'/data/user/tfutrell/it_anisotropy/{year}/{model}/{key}.npy'):
                    with open(f'/data/user/tfutrell/it_anisotropy/{year}/{model}/{key}.npy', 'rb') as file:
                        globals()[key] = np.load(file)
            
            # Set up weights dictionary
            WEIGHTS = {
                'H4a': Hweights,
                'GSF': Gweights
            }

            # Calculate Tiers
            quality_cuts = (laputop_zen < np.radians(55)) * (reco_pass == 1)
            offset = np.ceil((int(year) - 2011)/2)

            if int(year) < 2015:
                TIERS = {
                    'Tier 1': (3 <= hits) * (hits < 5) * (showerplane_zen < np.radians(55)),
                    'Tier 2': (5 <= hits) * (hits < 10 - offset) * quality_cuts,
                    'Tier 3': (10 - offset <= hits) * (hits < 17 - offset) * quality_cuts,
                    'Tier 4': (17 - offset <= hits) * quality_cuts
                }
            else:
                TIERS = {
                    'Tier 3': (10 - offset <= hits) * (hits < 17 - offset) * quality_cuts,
                    'Tier 4': (17 - offset <= hits) * quality_cuts
                }
                
            # Calculate then save the median and error
            for weight_name, weights in WEIGHTS.items():
                print(f'{weight_name}')
                for tier, tier_cut in TIERS.items():
                    print(f'    {tier}')
                    median = weighted_quantiles(energy[tier_cut], Gweights[tier_cut])
                    error = (abs(median - weighted_quantiles(energy[tier_cut], Gweights[tier_cut], 0.16))/1e6, abs(weighted_quantiles(energy[tier_cut], Gweights[tier_cut], 0.84) - median)/1e6)
                    
                    if weight_name == 'GSF':
                        MEDIANS[model][tier][year]['median'] = median/1e6
                        MEDIANS[model][tier][year]['error'] = error
                    
                    print(f'        median: {round(median/1e6, 3)}')
                    print(f'        error: {round(error[0], 3)}-{round(error[1], 3)}')

In [5]:
MODEL_COLORS = {
    'EPOS-LHC': 'blue',
    'QGSJET-II-04': 'black',
    'SIBYLL2.1': 'orange',
    'SIBYLL2.3': 'green',
    'SIBYLL2.3d': 'green',
}

CAPTHICK = {
    'Tier 1': 4,
    'Tier 2': 3,
    'Tier 3': 2,
    'Tier 4': 1
}

FMT = {
    'Tier 1': '-o',
    'Tier 2': '-h',
    'Tier 3': '-^',
    'Tier 4': '-D'
}

In [ ]:
for model, TIERS in MEDIANS.items():
    print(model)
    print('--------------------------------')
    for tier, YEARS in TIERS.items():
        print(tier)
        y = []
        y_err = ([], [])
        for year, MEDERR in YEARS.items():
            print(f'    {year}')
            for key, values in MEDERR.items():
                if key == 'median':
                    y.append(round(values, 3))
                else:
                    y_err[0].append(round(values[0], 3))
                    y_err[1].append(round(values[1], 3))
        print(f'        median-{y}')
        print(f'        uncertainty-{y_err}')
        print()

In [ ]:
fig, axs = plt.subplots(figsize=(13, 8))

for model, TIERS in MEDIANS.items():
    for tier, YEARS in TIERS.items():
        x = []
        y = []
        y_err = ([], [])
        for year, MEDERR in YEARS.items():
            x.append(year)
            for key, values in MEDERR.items():
                if key == 'median':
                    y.append(values)
                else:
                    y_err[0].append(values[0])
                    y_err[1].append(values[1])

        axs.errorbar(x, y, yerr=y_err, capsize=8, capthick=CAPTHICK[tier], fmt=FMT[tier], label=f'{model}-{tier}', c=MODEL_COLORS[model])

        '''if year == '2012':
            if tier == 'Tier 1':
                axs.errorbar(x, y, yerr=y_err, capsize=8, capthick=CAPTHICK[tier], fmt=FMT[tier], label=f'{model}', c=MODEL_COLORS[model])
            else: 
                axs.errorbar(x, y, yerr=y_err, capsize=8, capthick=CAPTHICK[tier], fmt=FMT[tier], c=MODEL_COLORS[model])
        elif model == 'SIBYLL2.3d':
            if tier == 'Tier 3':
                axs.errorbar(x, y, yerr=y_err, capsize=8, capthick=CAPTHICK[tier], fmt=FMT[tier], label=f'{model}', c=MODEL_COLORS[model])
            else: 
                axs.errorbar(x, y, yerr=y_err, capsize=8, capthick=CAPTHICK[tier], fmt=FMT[tier], c=MODEL_COLORS[model])
        else:
            axs.errorbar(x, y, yerr=y_err, capsize=8, capthick=CAPTHICK[tier], fmt=FMT[tier], c=MODEL_COLORS[model])'''
        axs.set_yscale("log", nonpositive='mask')

plt.xlabel('Year', size=16)
plt.ylabel('$\log_{10}$ Energy (PeV)', size=16)
plt.title(f'Median Energy per IceTop Tier per year', size=24)

plt.legend(loc='best', ncols=5)

plt.savefig(f'/data/user/tfutrell/it_anisotropy/yearly_median_e.pdf', format="pdf")